## Data creation

In [1]:
# Import libraries

from utilities import *

import rasterio
from osgeo import gdal
from osgeo import ogr
from rasterio.mask import mask
import os
from rasterio import plot
from rasterio.features import shapes
from rasterio import features

from glob import glob

import numpy as np
from shapely.geometry import box

import numpy as np
import xarray as xr
import geopandas as gpd
from geocube.api.core import make_geocube

import matplotlib.pyplot as plt

In [2]:
# Insert directories for input 

vec1_ds = "Inventories/Predicted/China/2017.shp" # Input destination to the Tn landslides
vec2_ds = "Inventories/Predicted/China/2014.shp" # Input destination to the Tn-1 landslides

subtracted_output_img = "Inventories/Subtracted/China/2017_2014.tif"
subtracted_output_vector = "Inventories/Subtracted/China/2017_2014.shp"

In [3]:
# Function to convert the polygons to raster images. 

def temporal_subtraction(vector1, vector2, size, output_path):

    """
    Arguments-
    
    (str): vector1: directory to the shapefile of the Tn landslide
    (str): vector2: directory to the shapefile of the Tn-1 landslide
    (integer): size: pixel size
    (str): output_path: set directory to where to save
    """
    v1 = gpd.read_file(vector1)
    v2 = gpd.read_file(vector2)

    raster1 = make_geocube(
                            vector_data=v1,
                            measurements=["raster_val"],
                            resolution=(-size, size),
                            fill=0
                           )
    
    raster2 = make_geocube(
                            vector_data=v2,
                            measurements=["raster_val"],
                            resolution=(-size, size),
                            fill=0
                           )
    # subtraction (Tn - Tn-1)
    minus = np.subtract(raster1,raster2).astype("uint16")
    
    return minus.rio.to_raster(output_path)

In [4]:
temporal_subtraction(vector1=vec1_ds, vector2=vec2_ds, size=3, output_path=subtracted_output_img)

In [5]:
def vectorize(input_image, output_vector):
    
    #  get raster data source
    open_image = gdal.Open(input_image)
    input_band = open_image.GetRasterBand(1)

    #  create output datasource
    output_shp = output_vector
    shp_driver = ogr.GetDriverByName("ESRI Shapefile")

    # create output file name
    output_shapefile = shp_driver.CreateDataSource(output_shp)
    new_shapefile = output_shapefile.CreateLayer(output_shp, srs=None)

    gdal.Polygonize(input_band, None, new_shapefile, -1, [], callback=None)
    new_shapefile.SyncToDisk()

In [8]:
vectorize(input_image=subtracted_output_img, output_vector=subtracted_output_vector)